In [37]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns

In [38]:
pd.set_option('display.max_columns', None) # Снимаем ограничение по клонкам
df = pd.read_csv('contest_train.csv', sep=',') # Загрузка файла
prediction_data = pd.read_csv('contest_test.csv', sep=',')
df.dropna(axis=1, how='any', inplace=True) # Удалеем колонны с None

In [39]:
df['targetBin'] = df["TARGET"].apply(lambda x: 0 if x in [0, 1] else 2).values # Для предсказания 2
myTrain = df.iloc[:15000, :] # На чем будем учиться
myTest  = df.iloc[15000:, :] # Для предсказания, для сравнения подходов

In [40]:
# Модель для предсказание сначала 2-к потом 1-ц

two  = myTrain[myTrain['targetBin'] == 2]
zero = myTrain[myTrain['targetBin'] == 0].head(1000)
myTrainBin = pd.concat([two, zero])

In [41]:
df_corr = myTrainBin.corr().unstack().reset_index()
hig_corr_features = df_corr[(np.abs(df_corr[0]) > 0.35) & (df_corr['level_0'] != df_corr['level_1'])]
features = hig_corr_features[hig_corr_features['level_0'] == 'targetBin']['level_1'].values[:-1]
len(features)

9

In [42]:

X = myTrainBin[features].values
y = myTrainBin['targetBin'].values

model_search_two = RandomForestClassifier()
model_search_two.fit(X, y)

# Результат предсказания 2
Xtest = myTest[features].values
predict = model_search_two.predict(Xtest)
myTest['tRF01'] = predict

C:\Users\PC\AppData\Local\Temp\ipykernel_10680\382573838.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  myTest['tRF01'] = predict


In [43]:
# Находим 1
one  = myTrain[myTrain['TARGET'] == 1]
zero = myTrain[myTrain['TARGET'] == 0].head(3500)
myTrainbin01 = pd.concat([one, zero])

In [44]:
df_corr = myTrainbin01.corr().unstack().reset_index()
hig_corr_features = df_corr[(np.abs(df_corr[0]) > 0.04) & (df_corr['level_0'] != df_corr['level_1'])]
features = hig_corr_features[hig_corr_features['level_0'] == 'TARGET']['level_1'].values[:-1]
len(features)

12

In [45]:

X = myTrainbin01[features].values
y = myTrainbin01['TARGET'].values

model_search_one = RandomForestClassifier(n_estimators=100)
model_search_one.fit(X, y)

Xtest = myTest[features].values
predict = model_search_one.predict(Xtest)
myTest['TresultBin'] = predict

C:\Users\PC\AppData\Local\Temp\ipykernel_10680\2549859849.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  myTest['TresultBin'] = predict


In [46]:
# объединяем результат работы поиска 1 и 2
result = []
list_ = list(myTest[['tRF01', 'TresultBin']].values)
for i, j in list_:
    result.append(2 if i == 2 else j)
myTest['TresultBin'] = result

C:\Users\PC\AppData\Local\Temp\ipykernel_10680\2865831538.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  myTest['TresultBin'] = result


In [47]:
print("Результат работы раздельно:")
print(classification_report(myTest['TARGET'], myTest['TresultBin']))

Результат работы раздельно:
              precision    recall  f1-score   support

           0       0.77      0.46      0.58      2360
           1       0.30      0.43      0.35       822
           2       0.18      0.71      0.29       208

    accuracy                           0.47      3390
   macro avg       0.42      0.53      0.41      3390
weighted avg       0.62      0.47      0.50      3390

